## 資料操作
:label:`sec_ndarray`
 
為了能夠完成各種資料操作，我們需要某種方法來儲存和操作資料。
通常，我們需要做兩件重要的事：（1）獲取資料；（2）將資料讀入電腦後對其進行處理。
如果沒有某種方法來儲存資料，那麼獲取資料是沒有意義的。
 
首先，我們介紹$n$維陣列，也稱為*張量*（tensor）。
使用過Python中NumPy計算套件的讀者會對本部分很熟悉。
無論使用哪個深度學習框架，它的*張量類*（在MXNet中為`ndarray`，
在PyTorch和TensorFlow中為`Tensor`）都與Numpy的`ndarray`類似。
但深度學習框架又比Numpy的`ndarray`多一些重要功能：
首先，GPU很好地支援加速計算，而NumPy僅支援CPU計算；
其次，張量類支援自動微分。
這些功能使得張量類更適合深度學習。
如果沒有特殊說明，本書中所說的張量均指的是張量類的實例。

## 入門

本節的目標是幫助讀者了解並運行一些在閱讀本書的過程中會用到的基本數值計算工具。
如果你很難理解一些數學概念或函式庫函數，請不要擔心。
後面的章節將通過一些實際的例子來回顧這些內容。
如果你已經具有相關經驗，想要深入學習數學內容，可以跳過本節。


(**首先，我們導入`torch`。請注意，雖然它被稱為PyTorch，但是程式碼中使用`torch`而不是`pytorch`。**)


In [1]:
import torch

[**張量表示一個由數值組成的陣列，這個陣列可能有多個維度**]。
具有一個軸的張量對應數學上的*向量*（vector）；
具有兩個軸的張量對應數學上的*矩陣*（matrix）；
具有兩個軸以上的張量沒有特殊的數學名稱。


首先，我們可以使用 `arange` 建立一個行向量 `x`。這個行向量包含以0開始的前12個整數，它們預設建立為整數。也可指定建立類型為浮點數。張量中的每個值都稱為張量的 *元素*（element）。例如，張量 `x` 中有 12 個元素。除非額外指定，新的張量將儲存在記憶體中，並採用基於CPU的計算。


In [2]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

[**可以透過張量的`shape`屬性來存取張量（沿著每個軸的長度）的*形狀***]
(~~和張量中元素的總數~~)。


In [3]:
x.shape

torch.Size([12])

如果只想知道張量中元素的總數，即形狀的所有元素乘積，可以檢查它的大小（size）。
因為這裡在處理的是一個向量，所以它的`shape`與它的`size`相同。


In [4]:
x.numel()

12

[**要想改變一個張量的形狀而不改變元素數量和元素值，可以調用`reshape`函數。**]
例如，可以把張量`x`從形狀為（12,）的行向量轉換為形狀為（3,4）的矩陣。
這個新的張量包含與轉換前相同的值，但是它被看成一個3行4列的矩陣。
要重點說明一下，雖然張量的形狀發生了改變，但其元素值並沒有變。
注意，通過改變張量的形狀，張量的大小不會改變。


In [5]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

我們不需要透過手動指定每個維度來改變形狀。
也就是說，如果我們的目標形狀是（高度,寬度），
那麼在知道寬度後，高度會被自動計算得出，不必我們自己做除法。
在上面的例子中，為了獲得一個3行的矩陣，我們手動指定了它有3行和4列。
幸運的是，我們可以透過`-1`來呼叫此自動計算出維度的功能。
即我們可以用`x.reshape(-1,4)`或`x.reshape(3,-1)`來取代`x.reshape(3,4)`。

有時，我們希望[**使用全0、全1、其他常量，或者從特定分布中隨機採樣的數字**]來初始化矩陣。
我們可以創建一個形狀為（2,3,4）的張量，其中所有元素都設置為0。程式碼如下：


In [6]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

同樣，我們可以創建一個形狀為`(2,3,4)`的張量，其中所有元素都設置為1。程式碼如下：


In [7]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

有時我們想透過從某個特定的機率分布中隨機取樣來得到張量中每個元素的值。
例如，當我們建構陣列來作為神經網路中的參數時，我們通常會隨機初始化參數的值。
以下程式碼建立一個形狀為（3,4）的張量。
其中的每個元素都從平均值為0、標準差為1的標準高斯分布（常態分布）中隨機取樣。


In [8]:
torch.randn(3, 4)

tensor([[ 0.9393, -1.4191, -1.3142,  1.2198],
        [ 0.7113, -1.0781, -0.1366,  0.5977],
        [-0.4874, -0.5705,  1.0437,  0.8175]])

我們還可以[**透過提供包含數值的Python列表（或嵌套列表），來為所需張量中的每個元素賦予確定值**]。
在這裡，最外層的列表對應於軸0，內層的列表對應於軸1。


In [9]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

## 運算子

我們的興趣不僅限於讀取資料和寫入資料。
我們想在這些資料上執行數學運算，其中最簡單且最有用的操作是*按元素*（elementwise）運算。
它們將標準純量運算子應用於陣列的每個元素。
對於將兩個陣列作為輸入的函數，按元素運算將二元運算子應用於兩個陣列中的每對位置對應的元素。
我們可以基於任何從純量到純量的函數來創建按元素函數。

在數學表示法中，我們將通過符號$f: \mathbb{R} \rightarrow \mathbb{R}$
來表示*一元*純量運算子（只接收一個輸入）。
這意味著該函數從任何實數（$\mathbb{R}$）映射到另一個實數。
同樣，我們通過符號$f: \mathbb{R}, \mathbb{R} \rightarrow \mathbb{R}$
表示*二元*純量運算子，這意味著該函數接收兩個輸入，並產生一個輸出。
給定同一形狀的任意兩個向量$\mathbf{u}$和$\mathbf{v}$和二元運算子$f$，
我們可以得到向量$\mathbf{c} = F(\mathbf{u},\mathbf{v})$。
具體計算方法是$c_i \gets f(u_i, v_i)$，
其中$c_i$、$u_i$和$v_i$分別是向量$\mathbf{c}$、$\mathbf{u}$和$\mathbf{v}$中的元素。
在這裡，我們通過將純量函數升級為按元素向量運算來生成向量值
$F: \mathbb{R}^d, \mathbb{R}^d \rightarrow \mathbb{R}^d$。

對於任意具有相同形狀的張量，
[**常見的標準算術運算子（`+`、`-`、`*`、`/`和`**`）都可以被升級為按元素運算**]。
我們可以在同一形狀的任意兩個張量上調用按元素操作。
在下面的例子中，我們使用逗號來表示一個具有5個元素的元組，其中每個元素都是按元素操作的結果。


In [10]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y  # **运算符是求幂运算

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

(**「按元素」方式可以應用更多的計算**)，包括像求冪這樣的一元運算子。


In [11]:
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

除了按元素計算外，我們還可以執行線性代數運算，包括向量點積和矩陣乘法。
我們將在 :numref:`sec_linear-algebra`中解釋線性代數的重點內容。

[**我們也可以把多個張量*連結*（concatenate）在一起**]，
把它們端對端地疊起來形成一個更大的張量。
我們只需要提供張量列表，並給出沿哪個軸連結。
下面的例子分別演示了當我們沿行（軸-0，形狀的第一個元素）
和按列（軸-1，形狀的第二個元素）連結兩個矩陣時，會發生什麼情況。
我們可以看到，第一個輸出張量的軸-0長度（$6$）是兩個輸入張量軸-0長度的總和（$3 + 3$）；
第二個輸出張量的軸-1長度（$8$）是兩個輸入張量軸-1長度的總和（$4 + 4$）。


In [12]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

有時，我們想[**通過*邏輯運算符*構建二元張量**]。
以`X == Y`為例：
對於每個位置，如果`X`和`Y`在該位置相等，則新張量中相應項的值為1。
這意味著邏輯語句`X == Y`在該位置處為真，否則該位置為0。


In [13]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

[**對張量中的所有元素進行求和，會產生一個單元素張量。**]


In [14]:
X.sum()

tensor(66.)

## 廣播機制
:label:`subsec_broadcasting`
 
在上面的部分中，我們看到了如何在相同形狀的兩個張量上執行按元素操作。
在某些情況下，[**即使形狀不同，我們仍然可以通過調用
*廣播機制*（broadcasting mechanism）來執行按元素操作**]。
這種機制的工作方式如下：

1. 通過適當複製元素來擴展一個或兩個數組，以便在轉換之後，兩個張量具有相同的形狀；
2. 對生成的數組執行按元素操作。

在大多數情況下，我們將沿著數組中長度為1的軸進行廣播，如下例子：


In [15]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

由於`a`和`b`分別是$3\times1$和$1\times2$矩陣，如果讓它們相加，它們的形狀不匹配。
我們將兩個矩陣*廣播*為一個更大的$3\times2$矩陣，如下所示：矩陣`a`將複製列，
矩陣`b`將複製行，然後再按元素相加。


In [16]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

## 索引和切片
 
就像在任何其他Python陣列中一樣，張量中的元素可以通過索引訪問。
與任何Python陣列一樣：第一個元素的索引是0，最後一個元素索引是-1；
可以指定範圍以包含第一個元素和最後一個之前的元素。
 
如下所示，我們[**可以用`[-1]`選擇最後一個元素，可以用`[1:3]`選擇第二個和第三個元素**]：


In [17]:
X[-1], X[1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

[**除了讀取外，我們還可以透過指定索引來將元素寫入矩陣。**]


In [18]:
X[1, 2] = 9
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  9.,  7.],
        [ 8.,  9., 10., 11.]])

如果我們想[**為多個元素賦值相同的值，我們只需要索引所有元素，然後為它們賦值。**]
例如，`[0:2, :]`訪問第1行和第2行，其中":"代表沿軸1（列）的所有元素。
雖然我們討論的是矩陣的索引，但這也適用於向量和超過2個維度的張量。


In [19]:
X[0:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

## 節省記憶體
 
[**執行一些操作可能會導致為新結果分配記憶體**]。
例如，如果我們用`Y = X + Y`，我們將取消引用`Y`指向的張量，而是指向新分配的記憶體處的張量。

在下面的例子中，我們用Python的`id()`函數演示了這一點，
它給我們提供了記憶體中引用物件的確切位址。
執行`Y = Y + X`後，我們會發現`id(Y)`指向另一個位置。
這是因為Python首先計算`Y + X`，為結果分配新的記憶體，然後使`Y`指向記憶體中的這個新位置。


In [20]:
before = id(Y)
Y = Y + X
id(Y) == before

False

這可能是不可取的，原因有兩個：

1. 首先，我們不想總是不必要地分配記憶體。在機器學習中，我們可能有數百兆的參數，並且在一秒內多次更新所有參數。通常情況下，我們希望原地執行這些更新；
2. 如果我們不原地更新，其他引用仍然會指向舊的記憶體位置，這樣我們的某些程式碼可能會無意中引用舊的參數。


幸運的是，(**執行原地操作**)非常簡單。
我們可以使用切片表示法將操作的結果分配給先前分配的陣列，例如`Y[:] = <expression>`。
為了說明這一點，我們首先創建一個新的矩陣`Z`，其形狀與另一個`Y`相同，
使用`zeros_like`來分配一個全$0$的區塊。


In [21]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 1916554384544
id(Z): 1916554384544


[**如果在後續計算中沒有重複使用`X`，
我們也可以使用`X[:] = X + Y`或`X += Y`來減少操作的記憶體開銷。**]


In [22]:
before = id(X)
X += Y
id(X) == before

True

## 轉換為其他Python物件


將深度學習框架定義的張量[**轉換為NumPy張量（`ndarray`）**]很容易，反之也同樣容易。
torch張量和numpy陣列將共享它們的底層記憶體，就地操作更改一個張量也會同時更改另一個張量。


In [23]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

要(**將大小為1的張量轉換為Python標量**)，我們可以呼叫`item`函數或Python的內建函數。


In [24]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

## 小結
 
* 深度學習儲存和操作資料的主要介面是張量（$n$維陣列）。它提供了各種功能，包括基本數學運算、廣播、索引、切片、記憶體節省和轉換其他Python物件。

## 練習

1. 執行本節中的程式碼。將本節中的條件陳述式`X == Y`更改為`X < Y`或`X > Y`，然後看看你可以得到什麼樣的張量。
1. 用其他形狀（例如三維張量）替換廣播機制中按元素操作的兩個張量。結果是否與預期相同？


[Discussions](https://discuss.d2l.ai/t/1747)


In [27]:
#1. 執行本節中的程式碼。將本節中的條件陳述式`X == Y`更改為`X < Y`或`X > Y`，然後看看你可以得到什麼樣的張量。
## 練習
import torch

# Define tensors X and Y
X = torch.arange(12, dtype=torch.float32).reshape((3, 4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

# Perform comparisons
result_eq = X == Y
result_lt = X < Y
result_gt = X > Y

result_eq, result_lt, result_gt

(tensor([[False,  True, False,  True],
         [False, False, False, False],
         [False, False, False, False]]),
 tensor([[ True, False,  True, False],
         [False, False, False, False],
         [False, False, False, False]]),
 tensor([[False, False, False, False],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]]))

In [28]:
# 1. 用其他形狀（例如三維張量）替換廣播機制中按元素操作的兩個張量。結果是否與預期相同？
# Re-initialize the environment and perform the operations again
import torch

# Define three-dimensional tensors X and Y for broadcasting
X_3d = torch.arange(12, dtype=torch.float32).reshape((3, 1, 4))
Y_3d = torch.tensor([[[2.0, 1, 4, 3]], [[1, 2, 3, 4]], [[4, 3, 2, 1]]])

# Perform operations using broadcasting
result_add = X_3d + Y_3d
result_mul = X_3d * Y_3d
result_sub = X_3d - Y_3d

result_add, result_mul, result_sub


(tensor([[[ 2.,  2.,  6.,  6.]],
 
         [[ 5.,  7.,  9., 11.]],
 
         [[12., 12., 12., 12.]]]),
 tensor([[[ 0.,  1.,  8.,  9.]],
 
         [[ 4., 10., 18., 28.]],
 
         [[32., 27., 20., 11.]]]),
 tensor([[[-2.,  0., -2.,  0.]],
 
         [[ 3.,  3.,  3.,  3.]],
 
         [[ 4.,  6.,  8., 10.]]]))

表明廣播機制在三維張量上正常運作，可進行加法、乘法、減法等按元素操作。